In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# carregamento de bibliotecas
import sys
import os

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.ingestion import cotacao_atual_etanol_html
from src.features.data import criar_tabela_spark, salvar_tabela_delta_spark
from src.features.processing import renomear_colunas, data_ingestao

In [0]:
# VARIÁVEIS DE DESTINO
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.precos_acucar_incremental_raw"

In [0]:
# captura da cotação atual do acucar
URL = "https://www.noticiasagricolas.com.br/cotacoes/sucroenergetico/acucar-cristal-cepea"

df_cotacao = cotacao_atual_etanol_html(URL)

In [0]:
# sanitizacao das colunas do dataframe
 cols_sanitizacao={
        'Vencimento':'Data',
         'Variação Diária %':'Variacao_Diaria_pct',
         'R$/Saca de 50 Kg':'Reais_por_Saca_de_50_Kg'}
 
 df_limpo = renomear_colunas(df_cotacao,cols_sanitizacao

In [0]:
# persistência da tabela na camada bronze
salvar_tabela_delta_spark(df_cotacao,mode='append', partitionby=['Data'],merge_schema=True, path=TABELA_INCREMENTAL_RAW)


In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.precos_acucar_incremental_raw
LIMIT 10